In [31]:
import os

In [32]:
!apt-get -qq update && apt-get -qq install cuda-11-8 > /dev/null

os.environ['PATH'] = '/usr/local/cuda-11.8/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [33]:
!pip install implicit

In [34]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import coo_matrix, csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import ndcg_at_k
from implicit.nearest_neighbours import bm25_weight

In [35]:
train = pd.read_csv('hse_train.csv')
train.head()

,user_id,item_id,timestamp
0,258671,74254,1511701649
1,258671,115615,1511841435
2,258671,176624,1512105022
3,240498,45484,1511605442
4,240498,39504,1511756830


In [36]:
train.shape

(4842338, 3)

In [37]:
train.timestamp = pd.to_datetime(train["timestamp"], unit='s')

In [38]:
train.timestamp.min(), train.timestamp.max()

(Timestamp('2017-11-24 16:00:00'), Timestamp('2017-12-02 23:59:59'))

In [39]:
max_timestamp = train['timestamp'].max()
train['days_ago'] = (max_timestamp - train['timestamp']).dt.days
train.head()

,user_id,item_id,timestamp,days_ago
0,258671,74254,2017-11-26 13:07:29,6
1,258671,115615,2017-11-28 03:57:15,4
2,258671,176624,2017-12-01 05:10:22,1
3,240498,45484,2017-11-25 10:24:02,7
4,240498,39504,2017-11-27 04:27:10,5


То есть собраны данные за неделю

In [40]:
ALL_USERS = train['user_id'].unique().tolist()
ALL_ITEMS = train['item_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

train['user_ids'] = train['user_id'].map(user_map)
train['item_ids'] = train['item_id'].map(item_map)

In [41]:
row = train['user_ids'].values
col = train['item_ids'].values
data = np.ones(train.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))

In [42]:
def to_user_item_coo(df, shape):
    """ Turn a dataframe with transactions into a COO sparse users x items matrix"""
    row = df['user_ids'].values
    col = df['item_ids'].values
    data = np.ones(df.shape[0])
    return coo_matrix((data, (row, col)), shape=shape)


def split_data(df, validation_days=1):
    """ Split a pandas dataframe into training and validation data, using <<validation_days>>
    """
    df_train = df[df['days_ago'] > validation_days]
    df_val = df[df['days_ago'] <= validation_days]
    print(f"Train shape: {df_train.shape} - Validation shape: {df_val.shape}")
    print(f"Train users: {df_train['user_ids'].nunique()} - Validation users: {df_val['user_ids'].nunique()}")
    print(f"Train items: {df_train['item_ids'].nunique()} - Validation items: {df_val['item_ids'].nunique()}")
    print(f"Validation days: {df_val['timestamp'].min()} - {df_val['timestamp'].max()}")
    print(f"Train days: {df_train['timestamp'].min()} - {df_train['timestamp'].max()}")
    return df_train, df_val

def get_val_matrices(df, validation_days=1):
    df_train, df_val = split_data(df, validation_days=validation_days)

    # Оставляем в df_val только пользователей, которые были в df_train
    train_users = set(df_train['user_ids'].unique())
    df_val = df_val[df_val['user_ids'].isin(train_users)]

    shape = (len(ALL_USERS), len(ALL_ITEMS))

    coo_train = to_user_item_coo(df_train, shape=shape)
    coo_val = to_user_item_coo(df_val, shape=shape)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()

    return {
        'coo_train': coo_train,
        'csr_train': csr_train,
        'csr_val': csr_val
    }


def validate(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension)
    for <<iterations>> over matrices and validate with NDCG@20
    """
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']

    model = AlternatingLeastSquares(factors=factors,
                                  iterations=iterations,
                                  regularization=regularization,
                                  random_state=42)
    model.fit(coo_train, show_progress=show_progress)

    ndcg20 = ndcg_at_k(model, csr_train, csr_val, K=20, show_progress=show_progress)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> NDCG@20: {ndcg20}")
    return ndcg20

In [43]:
matrices = get_val_matrices(train, validation_days=1)

Train shape: (3783404, 6) - Validation shape: (1058934, 6)
Train users: 667411 - Validation users: 329948
Train items: 179795 - Validation items: 167634
Validation days: 2017-12-01 00:00:00 - 2017-12-02 23:59:59
Train days: 2017-11-24 16:00:00 - 2017-11-30 23:59:59


In [44]:
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
%%time
best_ndcg20 = 0
regularization = 0.01
for factors in [50, 100, 500]:
    for iterations in [10, 15, 20]:
        ndcg20 = validate(matrices, factors, iterations, regularization, show_progress=True)
        if ndcg20 > best_ndcg20:
            best_ndcg20 = ndcg20
            best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
            print(f"Best NDCG@20 found. Updating: {best_params}")

100%|██████████| 295378/295378 [04:30<00:00, 1093.40it/s]


Factors:  50 - Iterations: 10 - Regularization: 0.010 ==> NDCG@20: 0.0034024333193364395
Best NDCG@20 found. Updating: {'factors': 50, 'iterations': 10, 'regularization': 0.01}


100%|██████████| 295378/295378 [04:15<00:00, 1155.65it/s]


Factors:  50 - Iterations: 15 - Regularization: 0.010 ==> NDCG@20: 0.003435213064699296
Best NDCG@20 found. Updating: {'factors': 50, 'iterations': 15, 'regularization': 0.01}


100%|██████████| 295378/295378 [04:37<00:00, 1065.21it/s]


...

Factors: 500 - Iterations: 10 - Regularization: 0.010 ==> NDCG@20: 0.00425750949217909
Best NDCG@20 found. Updating: {'factors': 500, 'iterations': 10, 'regularization': 0.01}


100%|██████████| 295378/295378 [14:36<00:00, 336.86it/s]


Factors: 500 - Iterations: 15 - Regularization: 0.010 ==> NDCG@20: 0.004259534714370213
Best NDCG@20 found. Updating: {'factors': 500, 'iterations': 15, 'regularization': 0.01}


100%|██████████| 295378/295378 [12:06<00:00, 406.85it/s]


Factors: 500 - Iterations: 20 - Regularization: 0.010 ==> NDCG@20: 0.004256413105545322
CPU times: total: 18h 56min 40s
Wall time: 3h 20min 12s


In [55]:
shape = (len(ALL_USERS), len(ALL_ITEMS))
train_coo = to_user_item_coo(train, shape)
train_csr = train_coo.tocsr()
model = AlternatingLeastSquares(factors=1000,
                                iterations=15,
                                regularization=0.01,
                                random_state=42,
                                use_gpu=True)
weighted_train = bm25_weight(coo_train, K1=100, B=0.8)
weighted_train = weighted_train.tocsr()
model.fit(weighted_train)

  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
reverse_user_map = {v: k for k, v in user_map.items()}
reverse_item_map = {v: k for k, v in item_map.items()}

In [20]:
recommendations = []

for user_ids in tqdm(range(train_csr.shape[0])):
    # Get recommendations for each user
    recommended = model.recommend(
        userid=user_ids,
        user_items=train_csr[user_ids],
        N=20,
        filter_already_liked_items=True
    )[0]
    original_user_id = reverse_user_map[user_ids]
    original_item_ids = [reverse_item_map[item_idx] for item_idx in recommended[:20]]
    for item_id in original_item_ids:
        recommendations.append({'user_id': original_user_id, 'items': item_id})
df_submission = pd.DataFrame(recommendations)

100%|██████████| 701981/701981 [44:59<00:00, 260.07it/s]


In [22]:
df_submission

,user_id,items
0,258671,88182
1,258671,71039
2,258671,100842
3,258671,15531
4,258671,168302
...,...,...
14039615,240935,47395
14039616,240935,13817
14039617,240935,30285
14039618,240935,114333


In [21]:
df_submission.to_csv('als2.csv', index=False)

# ALS. Бейзлайн 2

In [56]:
train_users = train['user_id'].unique()
train_items = train['item_id'].unique()
max_timestamp = train['timestamp'].max()
train['days_ago'] = (max_timestamp - train['timestamp']).dt.days
train['weight'] = 1 / (1 + train['days_ago'] / 30)

In [57]:
train.head()

,user_id,item_id,timestamp,days_ago,user_ids,item_ids,weight
0,258671,74254,2017-11-26 13:07:29,6,0,0,0.833333
1,258671,115615,2017-11-28 03:57:15,4,0,1,0.882353
2,258671,176624,2017-12-01 05:10:22,1,0,2,0.967742
3,240498,45484,2017-11-25 10:24:02,7,1,3,0.810811
4,240498,39504,2017-11-27 04:27:10,5,1,4,0.857143


In [58]:
#Кодируем user_id и item_id
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
train['user_id'] = user_encoder.fit_transform(train['user_id'])
train['item_id'] = item_encoder.fit_transform(train['item_id'])

In [ ]:
train['user_id'].nunique(), train['user_id'].max(), train['item_id'].nunique(), train['item_id'].max()

(701981, 701980, 180599, 180598)

In [ ]:
interactions = csr_matrix(
    (train['weight'].values,
     (train['user_id'], train['item_id'])),
    shape=(train['user_id'].max()+1, train['item_id'].max()+1)
)

In [ ]:
model = AlternatingLeastSquares(factors=100, iterations=15, regularization=0.01)
model.fit(interactions.T)

/usr/local/lib/python3.11/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.11676311492919922 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
item_counts = train['item_id'].value_counts()
popular_items = item_encoder.transform(item_counts.head(100).index.values)

recommendations = []

for user_id in tqdm(range(interactions.shape[0])):
    # Получаем рекомендации
    recommended = model.recommend(user_id, interactions[user_id], N=20, filter_already_liked_items=True)[0]
    # Преобразование обратно к оригинальным ID
    original_items = item_encoder.inverse_transform(recommended[:20])
    original_user_id = user_encoder.inverse_transform([user_id])[0]

    # Сохранение результатов
    for item_id in original_items:
        recommendations.append({'user_id': original_user_id, 'items': item_id})
df_submission = pd.DataFrame(recommendations)

100%|██████████| 701981/701981 [4:01:08<00:00, 48.52it/s]  


In [ ]:
df_submission

,user_id,items
0,0,137568
1,0,3665
2,0,28020
3,0,102173
4,0,177158
...,...,...
14039615,701980,58593
14039616,701980,171750
14039617,701980,154143
14039618,701980,110938


In [ ]:
df_submission.to_csv('als.csv', index=False)